In [4]:
import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent))

from katacv.utils.related_pkgs.utility import *
from katacv.utils.related_pkgs.jax_flax_optax_orbax import *
from katacv.yolov1.yolov1_pretrain import get_pretrain_state

In [9]:
print("Loading state...")
state = get_pretrain_state()
path = "/home/wty/Coding/models/YOLOv1PreTrain/YOLOv1PreTrain-0022-lite"  # YOLOv1PreTrain lite weights folder
weights = ocp.PyTreeCheckpointer().restore(path)
state = state.replace(params=weights['params'], batch_stats=weights['batch_stats'])
print("Read weights successfully!")

Loading state...
Read weights successfully!


In [10]:
import translators.server as ts
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
from PIL import Image
import numpy as np, io
from katacv.utils.imagenet.label2readable import label2readable
import time, base64

upload_button = widgets.FileUpload(description="上传图片", multiple=True)
output = widgets.Output()
pred_output = widgets.Output()

def display_html(text, base64_image):
    html_str = f"<div style='display:flex; align-items:center;'>"
    html_str += f"<div style='flex: 0.8; '>{text}</div>"
    html_str += f"<img src='data:image/png;base64,{base64_image}' style='height: 200px; margin-left: 5px;'>"
    html_str += "</div>"
    display(HTML(html_str))

def handle_upload(change):
    with output:
        clear_output()
        uploaded_files = upload_button.value
        if uploaded_files:
            for i, uploaded_file in enumerate(uploaded_files):
                img = Image.open(io.BytesIO(uploaded_file['content'])).convert('RGB')
                matrix = np.expand_dims(np.array(img.resize((224,224))), 0)
                text = f"图片 {i+1}"
                start_time = time.time()
                logit = state.apply_fn(
                    {'params': state.params, 'batch_stats': state.batch_stats},
                    matrix, train=False
                )[0]
                text += f" 预测用时 {time.time() - start_time:.2f}s"
                proba = jax.nn.softmax(jax.device_get(logit))
                top5_idxs = np.argsort(proba)[::-1][:5]
                for i in range(5):
                    idx = int(top5_idxs[i])
                    name = label2readable[idx]
                    p = proba[idx]
                    text += f"</br>pred{i+1}: {p*100:.2f}% {name, ts.baidu(name, to_language='cn')}"
                # html_str = "<font color='white'>" + html_str + "</font>"
                
                base64_image = base64.b64encode(uploaded_file['content']).decode('utf-8')
                display_html(text, base64_image)
                # display(HTML(html_str))
                # display(widgets.HTML(html_str))
                # display(img)
        else:
            print("请上传一张图片")
# 绑定上传按钮的事件处理函数
upload_button.observe(handle_upload, names='value')

# 显示上传按钮和输出区域
line1 = widgets.HBox([upload_button, pred_output])
widgets.VBox([line1, output])

Using region  server backend.

